This notebook is used for get statistics about methods that change garbage characteristic of the object and methods that don't. <br>
Expected result:
- what part of all methods are methods that don't change garbage characteristic
- what part of all methods are methods that do change garbage characteristic
- what part of method calls are call of methods that don't change garbage characteristic of the object
- what part of method calls are call of methods that do change garbage characteristic of the object

In [163]:
import re

# read
def read_program(filename):
    f =  open(filename, 'r')
    program = f.read()
    f.close();
    return program

In [139]:
class Scope():
    def __init__(self, parent_scope = None):
        self.parent_scope = parent_scope
        self.local_vars = dict()
        self.tracked_vars = dict()
        self.vars_to_delete = set()
    def get_parent(self):
        return self.parent_scope
    
    def is_being_tracked(self, var):
        tracked_vars = self.tracked_vars
        scope = self
        while tracked_vars.get(var, None) == None:
            if scope.get_parent():
                scope = scope.get_parent()
            else:
                return False
        return tracked_vars.get(var)
    
    def add_local_var(self, var):
        self.local_vars[var] = GC.does_not_change
        
    def change_gc(self, var, new_state):
        tracked_vars = self.tracked_vars
        scope = self
        while tracked_vars.get(var, None) == None:
            if scope.get_parent():
                scope = scope.get_parent()
            else:
                raise(Exception("Can't change gc of the variable {}".format(var)))
        tracked_vars[var] = new_state
        
    def make_trackable(self, var):
        # value is just a placeholder,
        # the real purpose to store the key
        self.tracked_vars[var] = 1
    
    # method to make a decision what variable is safe to delete
    # inside this scope
    def decide(self):
        pass

In [132]:
from enum import Enum
class GC(Enum):
    pass_forward = -1
    does_not_change = 1
    not_sure = 0
    
def init_methods_table(tree):
    # key - method
    # value - dict with key - param_name and value - pass_forward/doesn't_change/not_sure)
    methods_table = dict()
    for t in tree.types:
        if str(t) == 'ClassDeclaration':
            class_name = t.name
            for method in t.methods:
                param_table = dict()
                for param in method.parameters:
                    param_table[param.name] = GC.not_sure 
                methods_table[".".join([class_name, method.name])] = param_table
    return  methods_table

In [160]:
import javalang
from javalang.tree import *
program = read_program("program.txt")
tree = javalang.parse.parse(program)
methods_table = init_methods_table(tree)
methods_table

{'HelloWorld.main': {'args': <GC.not_sure: 0>, 'c': <GC.not_sure: 0>},
 'HelloWorld.print': {'a': <GC.not_sure: 0>},
 'HelloWorld.print2': {'a': <GC.not_sure: 0>}}

In [161]:
def process_class(class_obj):
    # first we need to init scope
    # scope for class consists only of fields
    scope = Scope()
    
    for field in class_obj.fields:
        for decl in field.declarators:
            scope.add_local_var(decl.name)
    return scope
    for method in class_obj.methods:
        process_method(method, class_obj, scope)
    
    
def process_method(method, class_object, class_scope):
    scope = Scope(class_scope)
    for param in method.parameters:
        scope.make_trackable(param.name)
        
    for command in method.body:
        handler = handlers.get(str(command), None) #some function from dict
        if handler:
            handler(command, scope)
        else:
            print("Error! Can't process statement of type", str(command))
            continue
    return scope
        
    
def process_statement(statement, scope):
    pass

def process_method_invocation(method_invocation, scope):
    pass

# always return False because doesn't change gc of the object
def process_literal_declaration(variable, scope):
    return False

# return True if member_reference is interesting_object
# otherwise False
def process_member_reference(member_reference, scope):
    return scope.is_being_tracked(member_reference.member) != None


def process_variable_declarator(obj, scope):
    handler = handlers.get(str(obj.initializer), None)
    if handler:
        if handler(obj.initializer, scope):
            scope.add_local_var(obj.name)
            scope.make_trackable(obj.name)
        else:
            scope.add_local_var(obj.name)
    else:
        print("Can't process", str(obj.initializer))
        

def process_class_creator(obj, scope):
    for arg in obj.arguments:
        handler = handlers.get(str(arg), None)
        if handler:
            if handler(arg, scope):
                return True
        else:
            raise("Can't process", obj)
    
def process_local_variable_declaration(declaration, scope):
    variables = declaration.children[3]
    for var in variables:
        process_variable_declarator(var, scope)

def process_return_statement(returnStatement, scope):
    pass
        
def get_type_of_parameter(param):
    return param.type.children[0]

def process_method_invocation(mi, scope):
    for arg in mi.arguments:
        if str(arg) == "MemberReference":
            if scope.is_being_tracked(var=arg.member):
                scope.change_gc(arg.member, GC.pass_forward)
                
def process_statement_expression(expr, scope):
    handler = handlers.get(str(expr.expression), None)
    if handler:
        handler(expr.expression, scope)
    else:
        print("Can't process", str(expr.expression))

handlers = dict()
handlers['LocalVariableDeclaration'] = process_local_variable_declaration
handlers['Literal'] = process_literal_declaration
handlers['MemberReference'] = process_member_reference
handlers['ClassCreator'] = process_class_creator
handlers["MethodInvocation"] = process_method_invocation
handlers['StatementExpression'] = process_statement_expression

In [162]:
scope = process_class(tree.types[0])
scope2 = process_method(tree.types[0].methods[1], tree.types[0], scope)
scope2.local_vars, scope2.tracked_vars

Can't process BinaryOperation


({'f': <GC.does_not_change: 1>, 'g': <GC.does_not_change: 1>},
 {'a': 1, 'g': <GC.pass_forward: -1>})

In [151]:
path = tree.types[0].methods[1].body[2].expression
print(path)
list(filter(lambda x: x[0] != '_', dir(path)))

MethodInvocation


['arguments',
 'attrs',
 'children',
 'filter',
 'member',
 'position',
 'postfix_operators',
 'prefix_operators',
 'qualifier',
 'selectors',
 'type_arguments']